# 02 - Using the OpenAI Library with Azure AD

This lab replicates the main lab content, but shows how to use the OpenAI library with Azure AD authentication instead of API keys.

First, we use the `import` statement to let our application know that we're going to be using the OpenAI library in our code.

In [ ]:
import openai

Next, we'll bring in the values from our `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found OpenAI API Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("OpenAI API Base Endpoint not found. Have you configured the .env file?")

The OpenAI library needs some configuration parameters set to allow it to work with the OpenAI service. We'll set the following parameters.

Parameter name | Description
--- | ---
`openai.api_type` | To use Azure AD authentication, we set this to `azure_ad`
`openai.api_key` | This is the value for our API key
`openai.api_base` | This is the resource endpoint for the OpenAI service we're using
`openai_api_version` | This is the version of the API we are going to use

In [ ]:
openai.api_type = 'azure_ad'
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

When using Azure AD authentication, there are essentially two changes that need to be made compared to when using an API key:

1. The **api type** needs to be set to `azure_ad`
2. The **api key** needs to contain a valid Azure AD token

Using the below code, we can authenticate using Azure AD with Device Code login and then collect the token to use later for authentication with the Azure OpenAI service.

In [ ]:
# Setup Azure AD authentication

from azure.identity import DeviceCodeCredential

default_credential = DeviceCodeCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
openai.api_key = token.token
print ("Using Azure AD authentication")

## Get the list of available Model Deployments

We can now call the `Deployment.list()` method of the OpenAI library.

In [ ]:
openai.Deployment.list()

You'll see that the result is the same JSON data that we got when we called the API directly.

## Send a prompt to Azure OpenAI using the OpenAI library

Ok, now that we have the list of deployment models, let's try to do a Completion.

We'll use the values from our `.env` file to configure this.

In [ ]:
COMPLETION_MODEL = os.getenv("OPENAI_COMPLETION_MODEL")
DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

As we've configured the OpenAI Python library in the previous steps, we can go ahead and call the completions method to generate a response. We'll call the `Completion.create()` method and pass in the name of our `model`, the `deployment_id` and the `prompt` we want to use.

In [ ]:
r = openai.Completion.create(
    model = COMPLETION_MODEL,
    deployment_id = DEPLOYMENT_ID,
    prompt = "The one thing I love more than anything else is ",
)

print(r)

Again, the response above should be the same JSON data that we got when we called the API directly, containing details of the model we called, the response that was generated and the token usage.

We could also just extract the generated response.

In [ ]:
print(r.choices[0].text)

## Summary

The OpenAI library provides a more concise way to work with the OpenAI API. Once we've set up the initial parameters, we don't need to send them each time as we need to do with a direct API call. It's also easier to add information such as prompts to the call, as we can pass those values in as part of the call to the OpenAI library methods instead of having to pass in JSON objects as part of the request body.

You can find more details about the completions API in the reference documentation:

https://platform.openai.com/docs/api-reference/completions/create

## Up Next

In the next lab we'll begin looking at AI orchestrators. Whereas the OpenAI library simplfies working with the OpenAI API, orchestrators take things to the next level!